In [1]:
# for correct relative imports
import sys; sys.path.append("../var_es_dgm")

In [2]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, TensorDataset

from diffusers import DDPMScheduler
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pprint

from sklearn.preprocessing import StandardScaler

from var_es_dgm import TimeGrad
from var_es_dgm.basic_models.deepvar import DeepVaR
from var_es_dgm.basic_models import HistoricalSimulation, VarCov
from var_es_dgm.stat_tests import generate_report
from var_es_dgm.utils import seed_everything, compute_individual_returns, compute_portfolio_returns, estimate_var_es_torch

In [3]:
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context("paper")

In [4]:
DATA_FOLDER = "../../data/"
df = pd.read_csv(DATA_FOLDER + "complete_stocks.csv")
df["Date"] = pd.to_datetime(df["Date"])

In [5]:
len(df["Ticker"].unique())

89

## 1% 

In [6]:
res_timeGrad = list()
res_deepvar = list()
res_hist = list()
res_varcov = list()
alpha=0.01
RANDOM_STATE = 12

for i in range(5):
    # one complete cycle
    seed_everything(RANDOM_STATE + i)
    n_stocks = 10
    tickers = np.random.choice(df["Ticker"].unique(), n_stocks, replace=False)
    weights = 1/n_stocks
    print("Portfolio = {0}".format(" + ".join([f"{weights} * {i}"for i in tickers])))
    df_copy = df.loc[df["Ticker"].isin(tickers)].copy(deep=True)

    df_returns = compute_individual_returns(df_copy)
    df_returns = compute_portfolio_returns(df_returns)

    univariate_target = df_returns["Return_Target"]

    univariate_target = univariate_target.values[1:]
    train_size = df_returns[df_returns.Date <= "2022-06-01"].index[-1] + 1
    test_size = len(univariate_target) - train_size
    train = univariate_target[:train_size]

    ss = StandardScaler()
    train_scaled = torch.tensor(ss.fit_transform(train.reshape(-1, 1)), dtype=torch.float32)

    seed_everything(RANDOM_STATE)
    context_size = 90
    num_train_samples = 3000
    train_data = torch.zeros(num_train_samples, context_size, train_scaled.shape[1])
    train_target = torch.zeros(num_train_samples, 1, train_scaled.shape[1])
    train_idx = np.random.choice(np.arange(context_size, train_scaled.shape[0]), num_train_samples, replace=False)
    
    for i in tqdm(range(num_train_samples)):
        idx = train_idx[i]
        train_context = train_scaled[idx-context_size:idx]
        target_obs = train_scaled[idx]
        train_data[i] = train_context
        train_target[i] = target_obs

    # Create DataLoader for ease of torch training
    train_loader = DataLoader(TensorDataset(train_data, train_target), batch_size=128, shuffle=False)


    temp = torch.tensor(ss.transform(univariate_target.reshape(-1, 1)))
    test_data_context = torch.zeros(test_size, context_size, temp.shape[1])
    test_data_real = torch.zeros(test_size, 1, temp.shape[1])
    for i in range(test_size):
        idx = i + train_size
        test_data_context[i] = temp[idx-context_size:idx]
        test_data_real[i] = temp[idx]

    seed_everything(RANDOM_STATE)
    sheduler = DDPMScheduler(num_train_timesteps=46, beta_end=0.35878774019831244, clip_sample=False)
    model = TimeGrad(1, 1, hidden_size=50, num_layers=2, scheduler=sheduler, num_inference_steps=46)
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0008985916117829954)
    device = "cuda"
    n_epochs = 31
    model.to(device);

    model.fit(train_loader, optimizer, n_epochs, device)


    seed_everything(RANDOM_STATE)
    VaR_TimeGrad = torch.zeros(test_data_real.shape[0])
    ES_TimeGrad = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_TimeGrad[i], ES_TimeGrad[i] = estimate_var_es_torch(model, test, alpha=alpha, n_samples=500, device=device)

    res_timeGrad.append(generate_report(test_data_real.flatten(), VaR_TimeGrad, ES_TimeGrad, alpha=alpha))

     # 2. Инициализация DeepVaR
    model_deepvar = DeepVaR(
        input_size=train_data.shape[2],  
        target_dim=1,  
        hidden_size=64,  
        num_layers=2,  
        dropout_rate=0.4  
    )

    optimizer = torch.optim.Adam(model_deepvar.parameters(), lr=1e-4)  
    model_deepvar.to(device)

    model_deepvar.fit(train_loader, optimizer, n_epochs=30, device=device)

    VaR_deepvar = torch.zeros(test_data_real.shape[0])
    ES_deepvar = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_deepvar[i], ES_deepvar[i] = estimate_var_es_torch(model_deepvar, test, alpha=alpha, n_samples=500, device=device)

    res_deepvar.append(generate_report(test_data_real.flatten(), VaR_deepvar, ES_deepvar, alpha=alpha))

    # Результаты для других моделей
    hist_sim = HistoricalSimulation(alpha=alpha)
    VaR_histSim = torch.zeros(test_data_real.shape[0])
    ES_histSim = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_histSim[i], ES_histSim[i] = hist_sim.predict(test)
    
    res_hist.append(generate_report(test_data_real.flatten(), VaR_histSim, ES_histSim, alpha=alpha))
    
    var_cov = VarCov(alpha=alpha)
    
    VaR_varCov = torch.zeros(test_data_real.shape[0])
    ES_varCov = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_varCov[i], ES_varCov[i] = var_cov.predict(test)
    
    res_varcov.append(generate_report(test_data_real.flatten(), VaR_varCov, ES_varCov, alpha=alpha))

    # Выводим результаты для всех моделей
    print("TimeGrad Results:", res_timeGrad[-1])
    print("DeepVaR Results:", res_deepvar[-1])
    print("Historical Simulation Results:", res_hist[-1])
    print("VarCov Results:", res_varcov[-1])

Portfolio = 0.1 * INTC + 0.1 * DHR + 0.1 * COP + 0.1 * MRK + 0.1 * HON + 0.1 * T + 0.1 * MSFT + 0.1 * AXP + 0.1 * PEP + 0.1 * SYK


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

Epochs:   0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

...

Epochs:   0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

TimeGrad Results: {'Kupicks POF': 0.33831157927013206, 'Haas TBF': 0.5618091006890652, 'Christoffersen Test': 0.6657300145970582, 'Kupicks Test': 0.33831157927013206, 'Acerbi Szekely 1': 1.911816954612732, 'Acerbi Szekely 2': 0.00028966920217499137}
DeepVaR Results: {'Kupicks POF': 0.001074877937832245, 'Haas TBF': 2.2394053831451066e-05, 'Christoffersen Test': 0.5939519624147305, 'Kupicks Test': 0.001074877937832245, 'Acerbi Szekely 1': 2.3808517456054688, 'Acerbi Szekely 2': 0.000721470161806792}
Historical Simulation Results: {'Kupicks POF': 0.9839089602677263, 'Haas TBF': 0.11777640619725117, 'Christoffersen Test': 0.7155219203146557, 'Kupicks Test': 0.9839089602677263, 'Acerbi Szekely 1': 1.9719154834747314, 'Acerbi Szekely 2': 0.00019918337056878954}
VarCov Results: {'Kupicks POF': 0.6124944699749673, 'Haas TBF': 0.35891287510350023, 'Christoffersen Test': 0.747017734035957, 'Kupicks Test': 0.6124944699749673, 'Acerbi Szekely 1': 1.91619074344635, 'Acerbi Szekely 2': 0.0001451659

In [7]:
print(np.array([list(i.values()) for i in res_timeGrad]).mean(axis=0))
print(np.array([list(i.values()) for i in res_deepvar]).mean(axis=0))
print(np.array([list(i.values()) for i in res_hist]).mean(axis=0))
print(np.array([list(i.values()) for i in res_varcov]).mean(axis=0))

[3.71117196e-01 3.23054730e-01 6.60331396e-01 3.71117196e-01
 1.96454506e+00 3.18742858e-04]
[4.80211429e-02 3.38680694e-03 5.93123588e-01 4.80211429e-02
 2.41081915e+00 7.50171713e-04]
[4.74905851e-01 3.58427983e-01 7.11934199e-01 4.74905851e-01
 2.05907185e+00 2.26448207e-04]
[6.53381145e-01            nan 7.53200902e-01 6.53381145e-01
 2.07649233e+00 1.56829669e-04]
